# Ejercicios de probabilidad

Los * indican ejercicios o preguntas particularmente desafiantes (¡pero no opcionales!).

In [ ]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import sys
import numpy as np
from math import *
from matplotlib import pylab as plt

## Ejercicio 1

Supongamos que tenemos dos cajas, una roja y otra azul. En la caja roja hay 2 manzanas y 6 naranjas, y en la azul hay 3 manzanas y una naranja (ver Figura).

<img src='images/ejercicio1-1.png' width=300px>

Suponemos ahora que elegimos al azar<sup>[1](#myfootnote1)</sup> una de las cajas y de esa caja elegimos al azar una fruta. Después de ver qué fruta es, la volvemos a meter en la caja de la que salió. Suponiendo que elegimos la caja roja un 40% de las veces y la azul el 60% restante:

a. ¿Cuál es la probabilidad de que saquemos una manzana?

b. Dado que elegimos una naranja, ¿cuál es la probabilidad de que la caja elegida haya sido la azul?

<u>Notas</u> <a name="myfootnote1">1</a>: ¿Pueden definir más precisamente que quiere decir que elegir algo al azar?

## Ejercicio 2<sup>[1](#myfootnote1)</sup>

Ahora supongamos que en lugar de dos cajas tenemos tres: una roja (<em>r</em>), una azul (<em>a</em>) y una verde (<em>v</em>). La caja <em>r</em> contiene 3 manzanas, 4 naranjas y 3 limas; la caja <em>a</em> contiene 1 manzana, 1 naranja y 0 limas; la caja <em>v</em> contiene 3 manzanas, 3 naranjas y cuatro limas. Si se elige una caja al azar con las siguientes probabilidades p(<em>r</em>) = 0.2, p(<em>b</em>) = 0.2, p(<em>g</em>) = 0.6, y todos los elementos de cada caja tienen la misma probabilida de ser elegidos, responder:

a. ¿cuál es la probabilidad de sacar una manzana?

b. ¿Si se sacó una naranja, cuál es la probabilidad de que provenga de la caja verde?

<u>Notas</u> <a name="myfootnote1">1</a>: Adaptado del Ejercicio 1.3 del libro de Bishop.

## Ejercicio 3 (guiado casi hasta el final)

Tomaremos un set de Datos de Student Alcohol Consumption de la UCI alojada por Kaggle. Pueden descargar el dataset de la clase de matemáticas aquí https://www.kaggle.com/uciml/student-alcohol-consumption#student-mat.csv y subirlo en google Colab, o ejecutar la celda de abajo. En el repo, ya esta alojado en el directorio datasets.

Para los que usan Google Colab, ejecuten esta celda una unica vez para descargar los datos. Si clonaron el repositorio usando git, no hace falta.


In [ ]:
if 'google.colab' in sys.modules:
    #Si estas en Google Colab, ejecuta esta celda una unica vez:
    !mkdir datasets
    !wget https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/datasets/student-mat.csv
    !mv student-mat.csv datasets

Primero que nada, carguemos el dataset usando pandas:

In [ ]:
df = pd.read_csv('datasets/student-mat.csv')
print("Cantidad de registros: {}".format(len(df)))
df.head()

Los datos son de una encuesta realizada a 395 alumnos de una clase de matemáticas, y tenemos muchos campos. Nos concentraremos en la siguiente pregunta: __¿Cual es la probabilidad de aprobar con una nota de 80% o más si falté 10 o más clases?__

Para hacerlo, crearemos unas columnas con 0 o 1 dependiendo de si cumplen las condiciones o no, así facilitaremos la tarea de contabilizar. La nota final esta en la columna G3, en una escala de 0 a 20. Las ausencias se contabilizan en absences.

In [ ]:
df['aprueba'] = np.where(df['G3']*5 >= 80, 1, 0)
df['falto_mucho'] = np.where(df['absences'] >= 10, 1, 0)

Para contar las columnas, nos será útil crear una columna de 1 constante, así contar es simplemente sumar dicho campo.

In [ ]:
df['cuenta'] = 1

Ahora nos deshacemos de todas las otras columnas:

In [ ]:
df = df[['aprueba','falto_mucho','cuenta']]
df.head()

Lo que sigue es transformar nuestra tabla. Queremos una tabla 2x2, la cuyos ejes sean el valor de "aprueba" y "falto mucho", y cuyos contenidos sea la cantidad de alumnos en cada categoría. Eso lo podemos conseguir utilizando el método pd.pivot_table usando la suma como función de agregación de la columna cuenta:

La función `pivot_table` toma un DataFrame y crea a partir de el otro que tenga como índices los valores de la columna indicada en `index=`, como columnas los valores de las columnas indicadas en `columns=`, y como valores a los de las columnas indicadas en `values=`. Si hay muchas coincidencias de valores para una misma celda, forma un vector. Para obtener un número en cada celda, seleccionamos una función de agregación con aggfunc= que se aplicará sobre los vectores. En caso de que hayan celdas vacías, se rellenarán usando el valor especificado en `fill_value=`.

En nuestro caso, como usamos de valores la columna cuenta que tiene solo números 1, la función de agregación para contar casos puede ser bien la de la suma np.sum. (Prueben también con len, la longitud del vector, que debería funcionar de igual modo).

In [ ]:
pd.pivot_table(
    df,
    values = 'cuenta',
    index = ['aprueba'],
    columns = ['falto_mucho'],
    aggfunc = np.sum,
    fill_value = 0)

Podemos ver entonces que, la cantidad de alumnos que falto poco y desaprobó es 277, que faltó mucho y desaprobó es 78, que faltó poco y aprobó es 35 y que faltó mucho y aprobó es 5, siendo la cantidad total de alumnos 395.

A partir de estos datos, obtenga la probabilidad de aprobar con una nota de 80% o más si falté 10 o más clases.

## *Ejercicio 4

En un programa de juegos de televisión, a un participante se le presentan tres puertas. Detrás de una de ellas se esconde un premio (digamos un auto). Si el participante elige la puerta detrás de la cual está el premio, se lo lleva. El participante elige una puerta, y el anfitrión (digamos Marley) abre una de las dos puertas restantes y revela que el premio no está detrás de esa puerta. A continuación, le ofrece al participante la posibilidad de cambiar de puerta.

a. ¿Qué le conviene hacer? Sin hacer ninguna cuenta, intetar argumentar a favor de alguna de las dos estrategias. (<b>Consejo</b>: pensar qué pasaría si en lugar de tres puertas, hubiera 10. O 100.)

b. Usar el teorema de Bayes para calcular la probabilidad ganar usando cada una de las estrategias.

*c. Simular la situación en la computadora, y confirmar el resultado del cálculo analítico. Ejecutar un número (1000?) de veces la simulación para cada una de las estrategias y contar las veces que se gana. 

d. Repetir el experimento varias veces y estudiar la distribución de veces que se gana con cada una de las estrategias. ¿Qué distribución de masa sería adecuada para modelar los resultados? ¿Por qué?

*e. Nuevo.* ¿Qué pasa si en lugar de ser el presentador el que abre la puerta detrás de la cual se ve que no hay premio, la puerta se abriera por algún proceso aleatorio (como un terremoto, o la caída de un elemento de iluminación del estudio de TV)? ¿Cómo cambia la respuesta? ¿Pueden? 

## *Ejercicio 5:

Supongamos que uno esta aburrido en una reunion con N personas y se quiere divertir. En ese caso, puede tratar de responder las siguientes preguntas:

a. ¿Cuál es la probabilidad de entre esas N personas haya al menos 2 que cumplen el mismo día?

b) Sabiendo esto, elije una persona al azar, y cumple el 3 de Marzo. ¿Qué probabilidad hay de que eligiendo otra persona al azar, esta cumpla el mismo día?

c. ¿Si en medio de la discusion entra otra persona al evento, que probabilidad tiene de cumplir el mismo día que alguno de los participantes?

d. Supongamos que usted es un desvelado y no sabe en que planeta está. Una manera de comprobarlo es ver si al menos 2 personas de los N participantes cumplen el mismo día. Sabiendo esto, ¿cómo inferiria cuánto dura el año en ese planeta? <b>Tip:</b> Utilicé la verosimulitud y el posterior.
